In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import IPython

Keras Tuner를 설치하고 가져옵니다.

In [27]:
!pip install -q -U keras-tuner
import kerastuner as kt

데이터 세트 다운로드 및 준비
    - 이 자습서에서는 Keras Tuner를 사용하여 Fashion MNIST 데이터 세트 에서 의류 이미지를 분류하는 기계 학습 모델에 가장 적합한 하이퍼 파라미터를 찾습니다.

In [28]:
# 데이터 로드
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [29]:
img_train.shape, label_train.shape

((60000, 28, 28), (60000,))

모델 정의

In [30]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))
    
    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                  metrics = ['accuracy'])
    
    return model

    튜너를 인스턴스화하고 하이퍼 튜닝을 수행합니다

    튜너를 인스턴스화하여 하이퍼 튜닝을 수행합니다. Hyperband Tuner에는 RandomSearch , Hyperband , BayesianOptimization 및 Sklearn 네 가지 튜너가 있습니다. 이 자습서에서는 하이퍼 밴드 튜너를 사용합니다.

    하이퍼 밴드 튜너를 인스턴스화하려면 하이퍼 모델, 최적화 할 objective 및 훈련 할 최대 max_epochs ( max_epochs )를 지정해야합니다.

In [31]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [32]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [33]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [34]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - ETA: 8:42 - loss: 2.1883 - accuracy: 0.18 - ETA: 20s - loss: 1.5613 - accuracy: 0.4721 - ETA: 11s - loss: 1.3144 - accuracy: 0.550 - ETA: 9s - loss: 1.1853 - accuracy: 0.591 - ETA: 7s - loss: 1.1046 - accuracy: 0.61 - ETA: 6s - loss: 1.0539 - accuracy: 0.63 - ETA: 6s - loss: 1.0195 - accuracy: 0.64 - ETA: 6s - loss: 0.9893 - accuracy: 0.65 - ETA: 5s - loss: 0.9689 - accuracy: 0.66 - ETA: 5s - loss: 0.9544 - accuracy: 0.67 - ETA: 6s - loss: 0.9429 - accuracy: 0.67 - ETA: 5s - loss: 0.9283 - accuracy: 0.68 - ETA: 5s - loss: 0.9130 - accuracy: 0.68 - ETA: 5s - loss: 0.9015 - accuracy: 0.68 - ETA: 5s - loss: 0.8923 - accuracy: 0.69 - ETA: 5s - loss: 0.8822 - accuracy: 0.69 - ETA: 5s - loss: 0.8716 - accuracy: 0.69 - ETA: 5s - loss: 0.8618 - accuracy: 0.70 - ETA: 5s - loss: 0.8483 - accuracy: 0.70 - ETA: 5s - loss: 0.8360 - accuracy: 0.71 - ETA: 4s - loss: 0.8242 - accuracy: 0.71 - ETA: 4s - loss: 0.8147 - accuracy: 0.71 - ETA: 4s - lo

1875/1875 [==============================] - ETA: 7s - loss: 0.2668 - accuracy: 0.87 - ETA: 4s - loss: 0.2867 - accuracy: 0.88 - ETA: 4s - loss: 0.2901 - accuracy: 0.89 - ETA: 4s - loss: 0.2983 - accuracy: 0.89 - ETA: 3s - loss: 0.3031 - accuracy: 0.89 - ETA: 3s - loss: 0.3054 - accuracy: 0.89 - ETA: 3s - loss: 0.3067 - accuracy: 0.89 - ETA: 3s - loss: 0.3087 - accuracy: 0.88 - ETA: 3s - loss: 0.3110 - accuracy: 0.88 - ETA: 3s - loss: 0.3125 - accuracy: 0.88 - ETA: 3s - loss: 0.3140 - accuracy: 0.88 - ETA: 3s - loss: 0.3150 - accuracy: 0.88 - ETA: 3s - loss: 0.3159 - accuracy: 0.88 - ETA: 3s - loss: 0.3167 - accuracy: 0.88 - ETA: 3s - loss: 0.3174 - accuracy: 0.88 - ETA: 3s - loss: 0.3180 - accuracy: 0.88 - ETA: 3s - loss: 0.3185 - accuracy: 0.88 - ETA: 3s - loss: 0.3189 - accuracy: 0.88 - ETA: 3s - loss: 0.3194 - accuracy: 0.88 - ETA: 3s - loss: 0.3199 - accuracy: 0.88 - ETA: 3s - loss: 0.3202 - accuracy: 0.88 - ETA: 3s - loss: 0.3205 - accuracy: 0.88 - ETA: 3s - loss: 0.3208 - accura

1875/1875 [==============================] - ETA: 7s - loss: 0.3435 - accuracy: 0.87 - ETA: 4s - loss: 0.3009 - accuracy: 0.89 - ETA: 3s - loss: 0.2893 - accuracy: 0.89 - ETA: 3s - loss: 0.2885 - accuracy: 0.89 - ETA: 3s - loss: 0.2894 - accuracy: 0.89 - ETA: 3s - loss: 0.2882 - accuracy: 0.89 - ETA: 3s - loss: 0.2872 - accuracy: 0.89 - ETA: 3s - loss: 0.2868 - accuracy: 0.89 - ETA: 3s - loss: 0.2870 - accuracy: 0.89 - ETA: 3s - loss: 0.2877 - accuracy: 0.89 - ETA: 3s - loss: 0.2885 - accuracy: 0.89 - ETA: 3s - loss: 0.2888 - accuracy: 0.89 - ETA: 3s - loss: 0.2889 - accuracy: 0.89 - ETA: 3s - loss: 0.2889 - accuracy: 0.89 - ETA: 3s - loss: 0.2887 - accuracy: 0.89 - ETA: 3s - loss: 0.2883 - accuracy: 0.89 - ETA: 3s - loss: 0.2881 - accuracy: 0.89 - ETA: 3s - loss: 0.2880 - accuracy: 0.89 - ETA: 3s - loss: 0.2879 - accuracy: 0.89 - ETA: 3s - loss: 0.2877 - accuracy: 0.89 - ETA: 2s - loss: 0.2875 - accuracy: 0.89 - ETA: 2s - loss: 0.2872 - accuracy: 0.89 - ETA: 2s - loss: 0.2870 - accura

1875/1875 [==============================] - ETA: 8s - loss: 0.2374 - accuracy: 0.90 - ETA: 3s - loss: 0.2432 - accuracy: 0.91 - ETA: 3s - loss: 0.2501 - accuracy: 0.91 - ETA: 3s - loss: 0.2520 - accuracy: 0.91 - ETA: 3s - loss: 0.2517 - accuracy: 0.91 - ETA: 3s - loss: 0.2506 - accuracy: 0.91 - ETA: 3s - loss: 0.2494 - accuracy: 0.91 - ETA: 3s - loss: 0.2490 - accuracy: 0.91 - ETA: 3s - loss: 0.2492 - accuracy: 0.91 - ETA: 3s - loss: 0.2493 - accuracy: 0.91 - ETA: 3s - loss: 0.2493 - accuracy: 0.91 - ETA: 3s - loss: 0.2491 - accuracy: 0.91 - ETA: 2s - loss: 0.2489 - accuracy: 0.91 - ETA: 2s - loss: 0.2488 - accuracy: 0.91 - ETA: 2s - loss: 0.2487 - accuracy: 0.91 - ETA: 2s - loss: 0.2487 - accuracy: 0.91 - ETA: 2s - loss: 0.2486 - accuracy: 0.91 - ETA: 2s - loss: 0.2485 - accuracy: 0.91 - ETA: 2s - loss: 0.2484 - accuracy: 0.91 - ETA: 2s - loss: 0.2483 - accuracy: 0.91 - ETA: 2s - loss: 0.2483 - accuracy: 0.91 - ETA: 2s - loss: 0.2483 - accuracy: 0.91 - ETA: 2s - loss: 0.2483 - accura

1875/1875 [==============================] - ETA: 11s - loss: 0.3092 - accuracy: 0.843 - ETA: 4s - loss: 0.2166 - accuracy: 0.911 - ETA: 4s - loss: 0.2219 - accuracy: 0.91 - ETA: 4s - loss: 0.2247 - accuracy: 0.91 - ETA: 3s - loss: 0.2247 - accuracy: 0.91 - ETA: 3s - loss: 0.2252 - accuracy: 0.91 - ETA: 3s - loss: 0.2252 - accuracy: 0.91 - ETA: 3s - loss: 0.2254 - accuracy: 0.91 - ETA: 3s - loss: 0.2260 - accuracy: 0.91 - ETA: 3s - loss: 0.2264 - accuracy: 0.91 - ETA: 3s - loss: 0.2267 - accuracy: 0.91 - ETA: 3s - loss: 0.2268 - accuracy: 0.91 - ETA: 3s - loss: 0.2269 - accuracy: 0.91 - ETA: 3s - loss: 0.2270 - accuracy: 0.91 - ETA: 3s - loss: 0.2270 - accuracy: 0.91 - ETA: 3s - loss: 0.2271 - accuracy: 0.91 - ETA: 3s - loss: 0.2271 - accuracy: 0.91 - ETA: 3s - loss: 0.2271 - accuracy: 0.91 - ETA: 3s - loss: 0.2270 - accuracy: 0.91 - ETA: 3s - loss: 0.2268 - accuracy: 0.91 - ETA: 3s - loss: 0.2266 - accuracy: 0.91 - ETA: 3s - loss: 0.2265 - accuracy: 0.91 - ETA: 3s - loss: 0.2266 - acc